## R Package TIMP showcase - global analysis of a 3 comp simulated decay

In [ ]:
import numpy as np

from lmfit import Parameters
import time

from glotaran_tools.specification_parser import parse_yml
from glotaran_models.kinetic import KineticSeparableModel

import matplotlib.pyplot as plt
%matplotlib notebook

fitspec = '''
type: kinetic

parameter: {}

compartments: [s1, s2, s3]

megacomplexes:
- label: mc1
  k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {{
      '("s1","s1")': 1,
      '("s2","s2")': 2,
      '("s3","s3")': 3,
    }}

initial_concentrations: []

irf:
  - label: irf1
    type: gaussian
    center: 4
    width: 5


datasets:
- label: dataset1
  type: spectral
  megacomplexes: [mc1]
  path: ''
  irf: irf1
'''
   

In [ ]:
initial_parameter = [123e-3, 234e-4, 567e-5, -0.05, 0.1]

initial_parameter = [123e-3, 234e-4, 567e-5, 0.28, 0.28]

times = np.concatenate([np.arange(-10, -1, 0.1).flatten(),
                        np.arange(-1, 10, 0.01).flatten(),
                        np.arange(10, 50, 1.5).flatten(),
                        np.arange(50, 1000, 15).flatten()])
wavenum = np.arange(12820, 15120, 4.6)

simparams = Parameters()
simparams.add("p1", 101e-3)
simparams.add("p2", 202e-4)
simparams.add("p3", 505e-5)
simparams.add("p4", 0.04)
simparams.add("p5", 0.5)
simparams.pretty_print()

In [ ]:
fitspec.format(initial_parameter)

In [ ]:
model = parse_yml(fitspec.format(initial_parameter))
print(model)

In [ ]:
fitmodel = KineticSeparableModel(model)
start = time.perf_counter()
data = fitmodel.eval(simparams, *times, **{'dataset':'dataset1', 
                                           'noise':True, 'noise_std_dev':0.000001,
                                           'dataset1_x': wavenum,
                                           'amplitudes':[1, 1, 1],
                                          'locations':[14700, 13515, 14180],
                                           'delta': [400,100,300]
                                           })

stop = time.perf_counter()
print("Simulating the data took: {:.2f}s".format(stop-start))
print("Data dimensions {:.0f}x{:.0f}".format(data.shape[0],data.shape[1]))
print("Total number of points {:.0f}".format(data.shape[0]*data.shape[1]))


In [ ]:
# DAS
DAS = fitmodel.coefficients(**{'dataset':'dataset1','dataset1_x': wavenum,
                                           'amplitudes':[1, 1, 1],
                                          'locations':[14700, 13515, 14180],
                                           'delta': [400,100,300]
                                           })
print(data[0,0])
print(wavenum.shape)

fig = plt.figure()
plt.plot(times, data[:, 300])

fig = plt.figure()
d1, = plt.plot(wavenum, DAS[:, 0], label=simparams['p1'].value)
d2, = plt.plot(wavenum, DAS[:, 1], label=simparams['p2'].value)
d3, = plt.plot(wavenum, DAS[:, 2], label=simparams['p3'].value)
plt.legend(handles=[d1, d2, d3])

In [ ]:
# Plot full trace

fig = plt.figure()
plt.xlabel('Time (ps)')
plt.ylabel('$Wavenumber\ [\ cm^{-1}\ ]$')
plt.pcolormesh(times, wavenum, data.T)
plt.show()

In [ ]:
#%time{fitmodel.fit(fitmodel.get_initial_fitting_parameter(), *times, **{"dataset1": data})}
start = time.perf_counter()
result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), *times, **{"dataset1": data})
stop = time.perf_counter()
result.best_fit_parameter.pretty_print()
reconstructed_data = result.eval(*times, **{"dataset1": data})
print("Fitting the data took: {:.2f}s".format(stop-start))

In [ ]:
# Plot residuals

fig = plt.figure()
plt.xlabel('Time (ps)')
plt.ylabel('$Wavenumber\ [\ cm^{-1}\ ]$')
plt.pcolormesh(times, wavenum, result.final_residual(*times, **{"dataset1": data}).T)
plt.show()

lsv, svals, rsv = result.final_residual_svd(*times, **{"dataset1": data})
print(svals.shape)

fig = plt.figure()
plt.plot(lsv[1, :], label="lsv")

fig = plt.figure()
plt.plot(svals[:20], label="sval")

fig = plt.figure()
plt.plot(rsv[1, :], label="rsv")

In [ ]:
# DAS
DAS = result.spectra(*times, **{'dataset':'dataset1', "dataset1": data})
print(DAS.shape)
fig = plt.figure()
d1, = plt.plot(wavenum, DAS[0, :], label=result.best_fit_parameter['p1'].value)
d2, = plt.plot(wavenum, DAS[1, :], label=result.best_fit_parameter['p2'].value)
d3, = plt.plot(wavenum, DAS[2, :], label=result.best_fit_parameter['p3'].value)
plt.legend(handles=[d1, d2, d3])



In [ ]:
# normalized DAS
nDAS = result.normalized_spectra(*times, **{'dataset':'dataset1', "dataset1": data})
print(DAS.shape)
fig = plt.figure()
d1, = plt.plot(wavenum, nDAS[0, :], label=result.best_fit_parameter['p1'].value)
d2, = plt.plot(wavenum, nDAS[1, :], label=result.best_fit_parameter['p2'].value)
d3, = plt.plot(wavenum, nDAS[2, :], label=result.best_fit_parameter['p3'].value)
plt.legend(handles=[d1, d2, d3])


In [ ]:
lsv, svals, rsv = result.svd(*times, **{"dataset1": data})
print(svals.shape)

fig = plt.figure()
plt.plot(times, lsv[:, :3], label="lsv")

fig = plt.figure()
plt.plot(svals[:20], label="sval")

fig = plt.figure()
plt.plot(rsv[0, :], label="rsv")

In [ ]:
lsv, svals, rsv = result.svd(*times, **{"dataset1": data})
print(svals.shape)

fig = plt.figure()
plt.plot(times, lsv[:, :3], label="lsv")